Using the following dataset, https://github.com/BestBuyAPIs/open-data-set; build 
an API with one endpoint that receives the “name” and “description” of a 
new product as input parameters, and outputs the “category” or “categories” that 
this new product should be in.

Expectations:
    • You have a data pipeline that handles the dataset.
    • You build a classifier that predicts the category(s) of a new product.
        ◦ You can decide if your model would output one or multiple labels.
        ◦ You don't need to spend lots of time comparing different models.
        ◦ You don't need to spend lots of time on trying to have the state of the art feature engineering.
    • You build one API endpoint that exposes the classifier as a solution to label new products.

In [24]:
import common
import pandas as pd

categories_path = 'open-data-set-master/categories.json'
products_path = 'open-data-set-master/products.json'
stores_path = 'open-data-set-master/stores.json'

categories = common.read_file(categories_path)
products = common.read_file(products_path)
stores = common.read_file(stores_path)

cat_df = pd.DataFrame.from_dict(categories)
prod_df = pd.DataFrame.from_dict(products)

prod_df[['cat1','cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7']] = pd.DataFrame(prod_df.category.tolist(), index= prod_df.index)


In [25]:
prod_df.head()

,sku,name,type,price,upc,category,shipping,description,manufacturer,model,url,image,cat1,cat2,cat3,cat4,cat5,cat6,cat7
0,43900,Duracell - AAA Batteries (4-Pack),HardGood,5.49,041333424019,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,Compatible with select electronic devices; AAA...,Duracell,MN2400B4Z,http://www.bestbuy.com/site/duracell-aaa-batte...,http://img.bbystatic.com/BestBuy_US/images/pro...,"{'id': 'pcmcat312300050015', 'name': 'Connecte...","{'id': 'pcmcat248700050021', 'name': 'Housewar...","{'id': 'pcmcat303600050001', 'name': 'Househol...","{'id': 'abcat0208002', 'name': 'Alkaline Batte...",None,None,None
1,48530,Duracell - AA 1.5V CopperTop Batteries (4-Pack),HardGood,5.49,041333415017,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,Long-lasting energy; DURALOCK Power Preserve t...,Duracell,MN1500B4Z,http://www.bestbuy.com/site/duracell-aa-1-5v-c...,http://img.bbystatic.com/BestBuy_US/images/pro...,"{'id': 'pcmcat312300050015', 'name': 'Connecte...","{'id': 'pcmcat248700050021', 'name': 'Housewar...","{'id': 'pcmcat303600050001', 'name': 'Househol...","{'id': 'abcat0208002', 'name': 'Alkaline Batte...",None,None,None
2,127687,Duracell - AA Batteries (8-Pack),HardGood,7.49,041333825014,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,Compatible with select electronic devices; AA ...,Duracell,MN1500B8Z,http://www.bestbuy.com/site/duracell-aa-batter...,http://img.bbystatic.com/BestBuy_US/images/pro...,"{'id': 'pcmcat312300050015', 'name': 'Connecte...","{'id': 'pcmcat248700050021', 'name': 'Housewar...","{'id': 'pcmcat303600050001', 'name': 'Househol...","{'id': 'abcat0208002', 'name': 'Alkaline Batte...",None,None,None
3,150115,Energizer - MAX Batteries AA (4-Pack),HardGood,4.99,039800011329,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,4-pack AA alkaline batteries; battery tester i...,Energizer,E91BP-4,http://www.bestbuy.com/site/energizer-max-batt...,http://img.bbystatic.com/BestBuy_US/images/pro...,"{'id': 'pcmcat312300050015', 'name': 'Connecte...","{'id': 'pcmcat248700050021', 'name': 'Housewar...","{'id': 'pcmcat303600050001', 'name': 'Househol...","{'id': 'abcat0208002', 'name': 'Alkaline Batte...",None,None,None
4,185230,Duracell - C Batteries (4-Pack),HardGood,8.99,041333440019,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",5.49,Compatible with select electronic devices; C s...,Duracell,MN1400R4Z,http://www.bestbuy.com/site/duracell-c-batteri...,http://img.bbystatic.com/BestBuy_US/images/pro...,"{'id': 'pcmcat312300050015', 'name': 'Connecte...","{'id': 'pcmcat248700050021', 'name': 'Housewar...","{'id': 'pcmcat303600050001', 'name': 'Househol...","{'id': 'abcat0208002', 'name': 'Alkaline Batte...",None,None,None


In [12]:
#Loading the data set - training data.
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)

In [14]:
print("\n".join(twenty_train.data[0].split("\n")[:3])) #prints first line of the first data file

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu


In [15]:
# Extracting features from text files
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(11314, 130107)

In [16]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 130107)

In [23]:
twenty_train.data[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [17]:
# Machine Learning
# Training Naive Bayes (NB) classifier on training data.
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [18]:
# Building a pipeline: We can write less code and do all of the above, by building a pipeline as follows:
# The names ‘vect’ , ‘tfidf’ and ‘clf’ are arbitrary but will be used later.
# We will be using the 'text_clf' going forward.
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

In [19]:
# Performance of NB Classifier
import numpy as np
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)
predicted = text_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.7738980350504514